# Projeto 1 - Ciência dos Dados

Nome: Letícia Coêlho Barbosa

Nome: Matheus Silva Melo de Oliveira

## Contextualização

------------------------------------------------------------

## Bibliotecas Utilizadas

------------------------------------------------------------

In [181]:
#!pip install emoji
#!pip install pysinonimos

Carregando algumas bibliotecas:

In [179]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 
import emoji
import pysinonimos.sinonimos as sn

from emoji import UNICODE_EMOJI

In [301]:
sn.Search('assistir').synonyms()

['ouvir',
 'comparecer',
 'observar',
 'presenciar',
 'testemunhar',
 'ver',
 'acompanhar',
 'assessorar',
 'atender',
 'auxiliar',
 'colaborar',
 'cooperar',
 'acudir',
 'ajudar',
 'amparar',
 'confortar',
 'olhar',
 'socorrer',
 'caber',
 'pertencer',
 'competir',
 'partejar',
 'morar',
 'habitar',
 'residir',
 'conservar-se',
 'estar',
 'ficar',
 'manter-se',
 'perdurar',
 'permanecer']

In [440]:
def dicio_r (palavra):
    
    nova=sn.Search(palavra).synonyms()   #Buscando sinônimos
    P_nova=0
    
    if nova != 404:
        
        for i in range(len(nova)):
        
            if (i != len(nova)-1):
                if (nova[i] in palavras_r) and (nova[i] not in lista_r):
                    P_nova=rel_relevantes[nova[i]]
                    return P_nova
                else:
                    continue

            else:
                return 1
    else:
        return 1
        

In [441]:
def dicio_i (palavra):
    
    nova=sn.Search(palavra).synonyms()   #Buscando sinônimos
    P_nova=0
    
    if nova != 404:
    
        for i in range(len(nova)):
            
            if (i != len(nova)-1):
            
                if (nova[i] in palavras_r) and (nova[i] not in lista_i):
                    P_nova=rel_relevantes[nova[i]]
                    return P_nova
                else:
                    continue
            
            else:
                return 1
    else:
        return 1
        

In [406]:
teste=rel_irrelevantes.index.tolist()

In [408]:
n= [x for x in palavras_i if x not in teste]
len(teste)

1353

In [409]:
len(palavras_i)

3961

In [307]:
#lista_r

In [308]:
#palavras_r

In [306]:
dicio_r('ouvir')

entender


0.0010584250635055038

In [180]:
for i in ['apresentação','dicionário']:
    
    apresentacao = sn.Search(i)
    sinonimos_de_apresentacao = apresentacao.synonyms()
    print(sinonimos_de_apresentacao)

['presença', 'ida', 'aparição', 'comparecimento', 'aparecimento', 'surgimento', 'vinda', 'aparência', 'figura', 'fisionomia', 'imagem', 'porte', 'aspecto', 'configuração', 'conformação', 'feitio', 'forma', 'formato', 'talhe', 'comunicação', 'adução', 'alegação', 'declaração', 'exposição', 'manifestação', 'relato', 'difusão', 'divulgação', 'propaganda', 'veiculação', 'abonação', 'indicação', 'proposição', 'proposta', 'recomendação', 'sugestão', 'atuação', 'demonstração', 'espetáculo', 'exibição', 'função', 'mostra', 'produção', 'sessão', 'show', 'anteâmbulo', 'introdução', 'preâmbulo', 'prefácio', 'prólogo', 'prolusão']
['glossário', 'léxico', 'vocabulário', 'léxicon', 'pai dos burros', 'tesouro', 'tira-dúvidas', 'tira-teimas', 'culto', 'douto', 'erudito', 'ilustrado', 'instruído', 'letrado', 'sabedor', 'sábio']


Confirmando diretório de trabalho

In [150]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Leticia\Documents\INSPER\2_Semestre\C_Dados\Projeto 1\TheMandalorian


## Banco de Dados

------------------------------------------------------------

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [151]:
filename = 'mandalorian.xlsx'

In [152]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,executivo da disney defendeu demissão de gina ...,0
1,@daredvevil nao seriooo melhor dia 😭 volta the...,1
2,preciso terminar mandalorian logo,1
3,toda vez que eu vejo o baby yoda usando a forç...,0
4,jon fraveau eu te amo obrigado por fazer the m...,1


In [153]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,the mandalorian é uma série com dinheiro e tec...,1
1,agora comecei mandalorian confiando novamente ...,1
2,@fepazo1 @gabriel_ns1999 @joelmsimmons @jurand...,1
3,eu assisti 3 minutos de mandalorian e emocionei,1
4,rt @anakinwar: já ta ...,0


## Classificador automático de sentimento

------------------------------------------------------------

Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


___
### Definindo Funções


In [322]:
#Limpando os tweets retirando pontos desnecessarios
def limpeza(tweet):
    punctuation = '[!-.:?;\n|]'                # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern,'', tweet)
    return text_subbed

#Limpando os tweets retirando emojis
def retira_emoji(content: str):
        modified = re.sub(emoji.get_emoji_regexp(), r"", content)
        modified = re.sub(':[^>]+:', '', modified)
        modified = re.sub('<[^>]+>', '', modified)
        return modified.strip() 
    
# Transformando tweets em lower case:
def lower(tweet):
    return tweet.lower()


# Concatenando tweets palavras+emojis limpos:
def concatenar(tweet):
    
    emojis=re.findall(emoji.get_emoji_regexp(),tweet)                   #Tornando os emojis em strings descritivas
    
    for i,emoji1 in enumerate(emojis):                              #Alguns emojis não possuem transcrição para português
            if emoji1 in UNICODE_EMOJI['pt']:
                emojis[i]=UNICODE_EMOJI['pt'][emoji1].replace(':',',')
            else:
                emojis[i]=UNICODE_EMOJI['en'][emoji1].replace(':',',')

                
    g=retira_emoji(tweet)
    g=lower(g)
    g=g.split()
    
    for i,word in enumerate(g):
        
        if 'rt' in word or 'https' in word or '@' in word:
            g[i]=g[i].replace(word,'null')
        
    g=[x for x in g if x!='null']
    frase_separada=','.join(g)
    emoji_transcrito=','.join(emojis)
    return frase_separada+','+emoji_transcrito

def frases_to_words (Serie_frases):
    
    texto=''
    
    for palavra in Serie_frases:
        texto+=str(palavra)                        #Criando uma string gigante com todas as palavras das frases
    
    lista_palavras=texto.split(',')                #Criando lista das palavras contidas na variável texto
    
    return pd.Series(lista_palavras)               #Retornando uma série de palavras contida nas frases

___
### Montando um Classificador Naive-Bayes



Considerando apenas as mensagens da planilha Treinamento:

In [216]:
#Transformando palavras em variáveis categóricas:

train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [217]:
#Aplicando funções de limpeza:

train['Clean']=train['Treinamento'].apply(limpeza)
train['Clean']=train['Clean'].apply(concatenar)

test['Clean']=test['Teste'].apply(limpeza)
test['Clean']=test['Clean'].apply(concatenar)

In [218]:
#Separando dados de treino por relevância:

train_relevantes=train[train['Relevancia']==1]
train_irrelevantes=train[train['Relevancia']==0]

In [186]:
#train.Treinamento[23]

In [187]:
#train.Clean[36]

In [197]:
#train_irrelevantes['Clean']

In [241]:
#Serie de palavras relevantes e irrelevantes no conjunto treino:

palavras_r= frases_to_words(train_relevantes['Clean'])
palavras_i= frases_to_words(train_irrelevantes['Clean'])

In [242]:
# Conferindo resultado:

palavras_r

0                                 nao
1                             seriooo
2                              melhor
3                                 dia
4                               volta
                    ...              
4719                              the
4720                      mandalorian
4721                                 
4722    mãos_juntas_pele_morena_clara
4723                                 
Length: 4724, dtype: object

In [469]:
#Criando texto com palavras relevantes:

texto_r=''

for palavra in train_relevantes['Clean']:
    texto_r+=str(palavra)
#print(texto_r)

In [244]:
#Criando texto com palavras irrelevantes:
texto_i=''

for palavra in train_irrelevantes['Clean']:
    texto_i+=str(palavra)


In [245]:
#Tranformando cada texto em uma Série de palavras:
#palavras_r=pd.Series(texto_r.split(','))
#palavras_i=pd.Series(texto_i.split(','))

In [246]:
#palavras_r

In [247]:
#palavras_i

In [248]:
# Frequência Relativa das palavras relevantes:
#rel_relevantes=palavras_r.value_counts(normalize=True)

In [249]:
#rel_relevantes.head(12)

In [250]:
#lista_r=['mandalorian','de','the','é','que','a','e','o','eu','da','mas','com']
#lista_r

#for i in lista_r:
    #print(rel_relevantes[i])

In [251]:
# Frequência Relativa das palavras irrelevantes:
#rel_irrelevantes=palavras_i.value_counts(normalize=True)

In [252]:
#rel_irrelevantes.head(18)

In [253]:
#lista_i=['mandalorian','de','the','e','o','a','que','do','é','eu','da','em','no','um','pra','não','com']

#for i in lista_i:
    #print(rel_irrelevantes[i])

In [254]:
# Frequência relativa das palavras:

rel_relevantes=palavras_r.value_counts(normalize=True)
rel_irrelevantes=palavras_i.value_counts(normalize=True)

In [255]:
# Conferindo resultado:
    
rel_relevantes.head()

mandalorian    0.057155
               0.041702
de             0.040644
the            0.031753
é              0.020322
dtype: float64

In [256]:
#Série representante do conjunto universo de todas as palavras:

palavras_i=palavras_i.tolist()
palavras_r=palavras_r.tolist()
#total=palavras_i+palavras_r
total=pd.Series(palavras_i+palavras_r)             
total

0                           executivo
1                                  da
2                              disney
3                            defendeu
4                            demissão
                    ...              
8680                              the
8681                      mandalorian
8682                                 
8683    mãos_juntas_pele_morena_clara
8684                                 
Length: 8685, dtype: object

In [257]:
#Frequência das palvras no total:

rel_total=total.value_counts(True)

In [258]:
#Conferindo resultado

rel_total

mandalorian    0.053771
de             0.039263
               0.029131
the            0.028094
o              0.021762
                 ...   
cabelinho      0.000115
acha           0.000115
desenvolveu    0.000115
rola           0.000115
chega          0.000115
Length: 2162, dtype: float64

### Montando as Probabilidades 

$$ P_{relevante}= \frac{Palavras_{relevantes}}{Total_{palavras}} $$

$$ P_{irrelevante}= \frac{Palavras_{irrelevantes}}{Total_{palavras}} $$

Onde, sabemos que:

$$ P_{relevante}+ P_{irrelevante}=1 $$

In [283]:
#Probabilidade tweet ser relevante ou irrelevante:

P_r=len(palavras_r)/len(total)
P_i=len(palavras_i)/len(total)

In [284]:
lista_r=['mandalorian','de','the','é','que','a','e','o','eu','da','mas','com']
lista_i=['mandalorian','de',' ','the','e','o','a','que','do','é','eu','da','em','no','um','pra','com'] #'não',

Ingenuidade do classificador Navie Bayes

$$ P(R|tweet)= \frac{P(tweet \cap R)}{P(tweet)} $$
$$ P(tweet|R)= \frac{P(tweet \cap R)}{P(R)} $$

$$ P(tweet \cap R) = P(R|tweet) P(tweet) $$
$$ P(tweet \cap R) = P(tweet|R) P(R) $$

Sendo a Probabiliadde de Navie Bayes:
    

$$ P(R|tweet)= \frac{P(tweet|R) P(R)}{P(tweet)} $$

A probabilidade $P(tweet|R)$, pode ser calculada considerando a independência entre as palavras, ou seja, a probabilidade de certa palavra aparecer não interfere na probabilidade de outras:

$$ P(tweet|R) = P(palavra1|R)P(palavra2|R)...P(palavraN|R)$$

Temos portayndo como porbabilidade de um tweet ser relevante:

$$ P(R|tweet) = \frac{P(palavra1|R)P(palavra2|R)...P(palavraN|R) P(R)}{P(tweet)}$$

Analogamento, para $P(I|tweet)$, teriamos:

$$P(I|tweet)=\frac{P(palavra1|I)P(palavra2|I)...P(palavraN|I) P(I)}{P(tweet)}$$

Portanto, a função Navie Bayes classificaria como relevante caso:

$$P(R|tweet)>P(I|tweet)$$

In [477]:
def P_tweet_r (tweet):
    
    c=1
    
    for palavra in tweet.split(','):
        if(palavra in palavras_r) and (palavra not in lista_r) :
            
            c*=rel_relevantes[palavra]
        elif palavra in rel_total:
            c*=rel_total[palavra]
            #c*=dicio_r(palavra)
        else:
            continue
           
    return c

In [478]:
def P_tweet_i (tweet):
    
    c=1
    
    for palavra in tweet.split(','):
        if(palavra in palavras_i) and (palavra not in lista_i):
            c*=rel_irrelevantes[palavra]
        
        elif palavra in rel_total: 
            c*=rel_total[palavra]
            #c*=dicio_i(palavra)
        else:
            continue
        
    return c

In [287]:
#P_tweet_r('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars')

In [288]:
#P_tweet_i('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars')

In [479]:
#Criando Nossa fução Naive_Bayes:

def Naive_Bayes (tweet):
    if(P_tweet_r(tweet)*P_r > P_tweet_i(tweet)*P_i):
        return 1
    else:
        return 0

In [461]:
 P_tweet_r('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars,alegria,Michel')

Michel


4.661988627906574e-51

In [462]:
P_tweet_i('dps,de,insistirem,mt,estou,começando,mandalorian,espero,q,realmente,seja,o,caso,de,salvação,do,star,wars,alegria, Michel')

 Michel


1.554710958099774e-54

In [480]:
#Criando uma nova coluna para armazenar resultado da nossa função Navie_Bayes

train['Classificador']=train.Clean.apply(Naive_Bayes)

In [481]:
train 

,Treinamento,Relevancia,Clean,Classificador
0,executivo da disney defendeu demissão de gina ...,0,"executivo,da,disney,defendeu,demissão,de,gina,...",0
1,@daredvevil nao seriooo melhor dia 😭 volta the...,1,"nao,seriooo,melhor,dia,volta,the,mandalorian,p...",1
2,preciso terminar mandalorian logo,1,"preciso,terminar,mandalorian,logo,",1
3,toda vez que eu vejo o baby yoda usando a forç...,0,"toda,vez,que,eu,vejo,o,baby,yoda,usando,a,forç...",0
4,jon fraveau eu te amo obrigado por fazer the m...,1,"jon,fraveau,eu,te,amo,obrigado,por,fazer,the,m...",1
...,...,...,...,...
495,"último ep de mandalorian, não estou preparada",1,"último,ep,de,mandalorian,não,estou,preparada,",1
496,minha prioridade agora é terminar the mandalorian,1,"minha,prioridade,agora,é,terminar,the,mandalor...",1
497,agora vou começar a ver the mandalorian 🙏🏼,1,"agora,vou,começar,a,ver,the,mandalorian,,mãos_...",1
498,"@jurandirfilho tem nem comparação né man, mand...",0,"tem,nem,comparação,né,man,mandalorian,",0


In [482]:
#Porgentagem de acerto no conjunto de Treino:

x=train.loc[(train['Classificador']==1)&(train['Relevancia']==1),:].shape[0]
y=train.loc[(train['Classificador']==0)&(train['Relevancia']==0),:].shape[0]
(x+y)/train.shape[0]

0.924

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [483]:
test['Classificador']=test.Clean.apply(Naive_Bayes)

In [484]:
x=test.loc[(test['Classificador']==1)&(test['Relevancia']==1),:].shape[0]
y=test.loc[(test['Classificador']==0)&(test['Relevancia']==0),:].shape[0]
(x+y)/test.shape[0]

0.56

In [485]:
# fp=test.loc[(test['Classificador']==0)&(test['Relevancia']==1),:].shape[0]
# tp=test.loc[(test['Classificador']==1)&(test['Relevancia']==1),:].shape[0]
# fn=test.loc[(test['Classificador']==1)&(test['Relevancia']==0),:].shape[0]
# tn=test.loc[(test['Classificador']==0)&(test['Relevancia']==0),:].shape[0]

# test['Classificador']=
pd.crosstab(test.Relevancia,test.Classificador,normalize=True)

Classificador,0,1
Relevancia,,
0,0.155556,0.391111
1,0.048889,0.404444


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**